
**Vectorized Observation**

Note: See Variable Definitions at bottom of page

Our previous HMM implementation only expected a single valued observation, however our CNN actually provides a probability distribution for each possible emotional state. In order to incorporate this probabilistic observation returned by the CNN we need to tweak the update step of the filtering process to accommodate this change. Filtering is a recursive algorithm used to find the probability of a state given a sensor observation of that state(observation step) and predicted probability of the prior state(recursive step). We also use a transition matrix which models the probability of transitioning from one state to another. 

In respects to this project, the observation step would be our CNN predicting the emotional state from a given frame. The 'recursive' step would be finding probability of the previous state by the same recursive filtering algorithm. In our case, we would be iterating from the first frame so these values do not need to be recursively recomputed. The observation matrix represents the probability of our CNN observation model predicting a certain value given a known actual emotional state. The transition matrix represents the probability of an emotional state transitioning from one to another between a single frame. The update step then combines these two probabilities using the transition and observation matrices, giving a prediction for the underlying hidden emotional state. The below diagram outlines this HMM structure:


![](https://github.com/ryanknauer/CPSC448/blob/master/Images/HMM_Diagram_1.png?raw=true)

The HMM update step consists of the following:  


$$
P(x_{t+1}) = \sum_{x_t}a_t(x_t)p(x_{t+1}|x_t)p(z_{t+1}|x_{t+1})
$$
Where $a_t(x_t)$ = recursive step; $p(x_{t+1}|x_t)$ = transition step;  $p(z_{t+1}|x_{t+1})$ = observation step

For the observation step, with a single value $z_{t+1}$ this would normally just be an index into our observation model:

For example if the observation matrix Y is: $$ Y  = \begin{bmatrix} 
0.8 & 0.1 & 0.1 \\
0.1 & 0.8 & 0.1 \\
0.1 & 0.1 & 0.8 
\end{bmatrix}
\quad$$ 


Then the probability of an observation z given state x is : $$ P(Z=z_t | X = x_t)  = Y[x_t,z_t]$$


So, if at any time $t$ we are given a state $x_t=1$ the probability of the observation $z_t = 1$  is then:  $$ P(Z=1| X = 1) = Y[1,1] = 0.8 $$

If instead $z_t$ is a multi-dimensional vector representing the observed probability of each emotional state we could then represent  $ P(z_t| x_t)  $ as weighted combination of each possible state given by:
$$
\sum_{i=0}^{len(z_t)} z_{t}[i]*P(Z=i | X = x_t)
$$
which is equivalent to taking the dot product of $ z_t$ and  $Y[:,x]$ if Y is our Observation Matrix. 


For example if our observation returned score of $$z_t = [0.3,0.4,0.3]$$ we would get: 
$$ P(z_t| x_t = 2) = (0.1)(0.3) + (0.8)(0.4) + (0.1)(0.3) =  0.38$$  

Finally, we can apply this vectorized observation model to the original HMM update step noted at the beginning of this section. Giving us:


$$
P(x_{t+1}) = \sum_{x_t}{a_t(x_t)p(x_{t+1}|x_t)\sum_{i=0}^{len(z_{t+1})} z_{t+1}[i]*P(Z=i | X = x_{t+1})}
$$

or:

$$
P(x_{t+1}) = \sum_{x_t}{a_t(x_t)p(x_{t+1}|x_t)(z_{t+1}Y[:,x_{t+1}])}
$$


This would likely be a  more accurate representation of the unconfident observation as it incorporates more of the available data. 


**Variable Reference**

z_t - observation of state at time t

x_t - actual state at time t

Z - Random variable representing observed state

X - Random variable representing actual state

Y - Observation matrix/model. 

For index z,x where z = observed state and x = actual state Y[z,x] = P(z|x) 






